In [6]:
import numpy as np
from environment.gobang import WrapperGoBang
from agent import Agent
import random
import os
from loguru import logger

In [7]:
class GoBangRandomAgent(Agent):
    def step(self, obs, explore=True, legal_actions=None):
        return {'action': random.choice(legal_actions)}

In [8]:
def do_exp(agent, gobang_n, epochs, test_n, test_freq, result_path):
    random_agent = GoBangRandomAgent(None)
    loss_episode = []
    reward_episode = []
    win_ratio = []
    best_win_ratio = -np.inf
    env = WrapperGoBang(gobang_n)
    for i in range(epochs):
        loss_list, reward_sum = agent.train(env)
        logger.info(f'Episode {i}/{epochs}, average loss: {np.mean(loss_list) if loss_list else -1}, reward sum: {reward_sum}')
        if loss_list:
            loss_episode.append(np.array([i, np.mean(loss_list)]))
            reward_episode.append(np.array([i, reward_sum]))
        
        if i % test_freq == 0:
            win = 0
            for _ in range(test_n):
                win += agent.test(env, random_agent, display=False)
            logger.info(f'Episode {i}/{epochs}, win {win}/{test_n}')
            ratio = 1.0 * win / test_n
            win_ratio.append(np.array([i, ratio]))
            if ratio > best_win_ratio:
                agent.save()
    np.save(os.path.join(result_path, 'loss.npy'), np.array(loss_episode))
    np.save(os.path.join(result_path, 'reward.npy'), np.array(reward_episode))
    np.save(os.path.join(result_path, 'ratio.npy'), np.array(win_ratio))    
    

# 1. DQN Agent

In [9]:
from agent import DQNAgent
from agent.model import ConvQModel
from agent.configuration import DQNConfig
from agent.exploration import LinearExploration

In [10]:
eval_model = ConvQModel(
    in_channels=2,
    in_h=7,
    in_w=7,
    hidden_channels=32,
    output_dim=49,
    dueling=False
)
target_model = ConvQModel(
    in_channels=2,
    in_h=7,
    in_w=7,
    hidden_channels=32,
    output_dim=49,
    dueling=False
)
exploration = LinearExploration(
    init_epsilon=1.0,
    min_epsilon=0.1,
    epsilon_decay=0.995
)
config = DQNConfig(
    eval_model=eval_model,
    target_model=target_model,
    ckpt_path='./dqn_gobang_n7.ckpt',
    batch_size=32,
    lr=0.001,
    gamma=0.99,
    target_replace_frequency=100,
    capacity=5000,
    max_grad_norm=1.0,
    exploration=exploration
)
agent = DQNAgent(config)
do_exp(agent, 7, 5000, 50, 50, './experiment/dqn_gobang_n7')

2023-05-05 04:45:52.858 | INFO     | __main__:do_exp:10 - Episode 0/5000, average loss: -1, reward sum: -26
2023-05-05 04:46:13.380 | INFO     | __main__:do_exp:19 - Episode 0/5000, win 32/50
2023-05-05 04:46:15.941 | INFO     | __main__:do_exp:10 - Episode 1/5000, average loss: 0.09544651653114412, reward sum: -47
2023-05-05 04:46:17.658 | INFO     | __main__:do_exp:10 - Episode 2/5000, average loss: 0.015170754460088726, reward sum: -27
2023-05-05 04:46:20.244 | INFO     | __main__:do_exp:10 - Episode 3/5000, average loss: 0.052820717662477264, reward sum: -45
2023-05-05 04:46:23.099 | INFO     | __main__:do_exp:10 - Episode 4/5000, average loss: 0.02179505278272583, reward sum: -48.2
2023-05-05 04:46:24.861 | INFO     | __main__:do_exp:10 - Episode 5/5000, average loss: 0.0095343595069128, reward sum: -26
2023-05-05 04:46:27.590 | INFO     | __main__:do_exp:10 - Episode 6/5000, average loss: 0.0633905636094769, reward sum: -38
2023-05-05 04:46:29.776 | INFO     | __main__:do_exp:10 

In [11]:
eval_model = ConvQModel(
    in_channels=2,
    in_h=11,
    in_w=11,
    hidden_channels=32,
    output_dim=121,
    dueling=False
)
target_model = ConvQModel(
    in_channels=2,
    in_h=11,
    in_w=11,
    hidden_channels=32,
    output_dim=121,
    dueling=False
)
exploration = LinearExploration(
    init_epsilon=1.0,
    min_epsilon=0.1,
    epsilon_decay=0.995
)
config = DQNConfig(
    eval_model=eval_model,
    target_model=target_model,
    ckpt_path='./dqn_gobang_n11.ckpt',
    batch_size=32,
    lr=0.001,
    gamma=0.99,
    target_replace_frequency=100,
    capacity=5000,
    max_grad_norm=1.0,
    exploration=exploration
)
agent = DQNAgent(config)
do_exp(agent, 11, 5000, 50, 50, './experiment/dqn_gobang_n11')

2023-05-05 05:38:06.653 | INFO     | __main__:do_exp:10 - Episode 0/5000, average loss: 0.07145430048281667, reward sum: -105
2023-05-05 05:39:09.565 | INFO     | __main__:do_exp:19 - Episode 0/5000, win 19/50
2023-05-05 05:39:12.008 | INFO     | __main__:do_exp:10 - Episode 1/5000, average loss: 0.04061470654053259, reward sum: -61
2023-05-05 05:39:13.503 | INFO     | __main__:do_exp:10 - Episode 2/5000, average loss: 0.022661257765319246, reward sum: -35
2023-05-05 05:39:16.376 | INFO     | __main__:do_exp:10 - Episode 3/5000, average loss: 0.05120222004342255, reward sum: -62
2023-05-05 05:39:20.535 | INFO     | __main__:do_exp:10 - Episode 4/5000, average loss: 0.04642143054215383, reward sum: -82
2023-05-05 05:39:24.715 | INFO     | __main__:do_exp:10 - Episode 5/5000, average loss: 0.039119385353988255, reward sum: -84
2023-05-05 05:39:29.662 | INFO     | __main__:do_exp:10 - Episode 6/5000, average loss: 0.04787604316061161, reward sum: -104
2023-05-05 05:39:33.585 | INFO     | 